# Cuaderno Jupyter realizado por Eduardo J. Barrios

La primera celda ejecutable de este cuaderno está intensionadamente escrita para que puedas usar este cuaderno con código [Julia](https://julialang.org/) en  [Google Colab](https://colab.research.google.com/?hl=es).





## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


Ponemos a mano los elementos de los paquetes instalados

In [2]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")
using JuMP, GLPK, Random, Test

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


# Introducción




Este cuaderno Jupyter ha sido desarrollado por Eduardo José Barrios García para la asignatura de Ingeniería Logística. El objetivo del cuaderno es abordar problemas de asignación 3-dimensional, tanto planar como axial. Se han utilizado modelos matemáticos previamente expuestos en el cuaderno de Google Colab del profesor, pero con modificaciones para aportar un enfoque único.

Este cuaderno muestra cómo se pueden resolver problemas de asignación 3-dimensional utilizando modelos matemáticos y el lenguaje de programación Julia. Se espera que este cuaderno sirva como una valiosa referencia para futuras investigaciones y aplicaciones prácticas.






# Problemas de asignación
 Para empezar a proceder con los problemas de asignación,deberemos dar algunas consideraciones previas antes, la primera de ellas, será añadir Gurubi para su realización:

In [ ]:
using Gurobi

## Problema de asignación 3-dimensional

### Introducción a los Problemas 3D
Los problemas de asignación en tres dimensiones (3D) son una extensión natural de los problemas de asignación en dos dimensiones (2D). En estos problemas, no solo se consideran variables en el plano cartesiano (x, y), sino que también se introduce una tercera dimensión (z). Esto puede tener aplicaciones en áreas como la logística, donde se deben asignar recursos en un espacio tridimensional, como un almacén.

### Consideraciones Específicas
- Complejidad Computacional: Resolver problemas en 3D generalmente implica un mayor costo computacional que en 2D debido al aumento en la cantidad de variables y restricciones.

- Representación Geométrica: La visualización de soluciones es más compleja. Herramientas de visualización en 3D pueden ser necesarias para entender completamente el problema y sus soluciones.

- Restricciones Adicionales: Puede haber restricciones adicionales relacionadas con la tercera dimensión, como limitaciones de altura o profundidad.

- Precisión en los Datos: La precisión requerida para las coordenadas en 3D es generalmente más alta, lo que puede requerir más datos y, posiblemente, sensores más precisos para capturar estos datos.

- Optimización Multiobjetivo: En muchos casos, podría haber múltiples objetivos a optimizar, como el costo, el tiempo y el uso del espacio, lo que complica aún más el problema.

### Ejemplo de Código en Julia para Problema 3D
A continuación se presenta un simple ejemplo en Julia que se extiende a 3 dimensiones. Añadiendo primeramente que paquetes vamos a usar:



In [ ]:
using Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")

Una vez intaladas, como hicimos en la parte de arriba, desarrollaamos el modelo:

In [ ]:
using JuMP
using GLPK

# Crear el modelo
model_3D = Model(GLPK.Optimizer)

# Definir variables de decisión
@variable(model_3D, x >= 0)
@variable(model_3D, y >= 0)
@variable(model_3D, z >= 0)

# Agregar restricciones
@constraint(model_3D, x + y + z <= 12)
@constraint(model_3D, x + 2y <= 10)
@constraint(model_3D, 2x + z <= 8)

# Establecer la función objetivo
@objective(model_3D, Max, 3x + 4y + 2z)

# Resolver el modelo
optimize!(model_3D)

# Mostrar los resultados
println("Estado de la solución: ", termination_status(model_3D))
println("Valor objetivo óptimo: ", objective_value(model_3D))
println("Valor óptimo de x: ", value(x))
println("Valor óptimo de y: ", value(y))
println("Valor óptimo de z: ", value(z))

Consideremos ahora $n$ profesores, $n$ asignaturas, y $n$ aulas. Asumamos conocer el coste $c_{ijk}$ de colocar al profesor $i$ impartiendo la asignatura $j$ en el aula $k$. Obviamente queremos que cada profesor imparta una asignatura única en un aula única. Igualmente queremos que cada asignatura la imparta un profesor en algún aula, y que cada aula tenga a un profesor impartiendo una asignatura. Se busca la asignación con menor coste total.

He procedido a cambiar el prblema propuesto por el profesor, usando el mismo fondo del problema, pero haciendo toda su estructura más legible.

In [ ]:
using Random  # Importar el módulo Random para utilizar funciones de generación de números aleatorios

N = 7  # Cambiado a 7: número de profesores, cursos y salones
Random.seed!(5678)  # Semilla para obtener diferentes números aleatorios

# Generar una matriz tridimensional de "precios" aleatorios
# Las dimensiones son N x N x N, representando, por ejemplo, Profesores x Cursos x Salones
# Los precios son números aleatorios entre 10 y 110
prices = rand(10:110, N, N, N)



ste es un modelo de optimización para un problema de asignación en un contexto educativo, pero que puede adaptarse a diversos escenarios. Imaginemos que tenemos NumEntities profesores, NumEntities cursos y NumEntities aulas. El objetivo es asignar a cada profesor exactamente un curso y una aula de manera que minimicemos un "costo" total asociado a esas asignaciones.

La variable assignmentMatrix[profIndex, courseIndex, roomIndex] es una variable binaria tridimensional. Tomará el valor de 1 si el profesor identificado con profIndex se asigna al curso identificado con courseIndex en la aula identificada con roomIndex. De lo contrario, su valor será 0.

La matriz cost tiene los "costos" asociados con asignar el profesor profIndex al curso courseIndex en el aula roomIndex. Este costo puede representar distintas métricas según el contexto: distancia, tiempo, recursos, entre otros.

Las restricciones del modelo aseguran tres cosas importantes:

Cada profesor (profIndex) se asigna a un solo curso y una sola aula.
Cada curso (courseIndex) tiene exactamente un profesor y se lleva a cabo en una sola aula.
En cada aula (roomIndex) se realiza exactamente un curso con un solo profesor.
Finalmente, la función objetivo busca minimizar el costo total de todas las asignaciones posibles, considerando las restricciones anteriores.

In [ ]:
using Gurobi  # Importar el módulo Gurobi para optimización
using JuMP  # Importar el módulo JuMP para modelado matemático

NumEntities = 7  # Número de profesores, cursos y aulas

# Crear un nuevo modelo de optimización utilizando Gurobi como optimizador
optimizationModel = Model(Gurobi.Optimizer)

# Silenciar la salida del optimizador para evitar imprimir mensajes en la consola
set_silent(optimizationModel)

# Definir la variable de decisión 'assignmentMatrix'. Es una variable binaria tridimensional de tamaño NumEntities x NumEntities x NumEntities
@variable(optimizationModel, assignmentMatrix[1:NumEntities, 1:NumEntities, 1:NumEntities], Bin)

# Establecer la función objetivo para minimizar el costo total.
@objective(optimizationModel, Min, sum(cost .* assignmentMatrix))

# Restricciones para asegurar que cada profesor (profIndex) se asigne a un solo curso y aula.
@constraint(optimizationModel, prof[profIndex=1:NumEntities], sum(assignmentMatrix[profIndex, :, :]) == 1)

# Restricciones para asegurar que cada curso (courseIndex) tenga un solo profesor y se realice en una sola aula.
@constraint(optimizationModel, course[courseIndex=1:NumEntities], sum(assignmentMatrix[:, courseIndex, :]) == 1)

# Restricciones para asegurar que cada aula (roomIndex) tenga un solo curso y profesor.
@constraint(optimizationModel, room[roomIndex=1:NumEntities], sum(assignmentMatrix[:, :, roomIndex]) == 1)

# Imprimir el modelo para verificar que se ha configurado correctamente
println(optimizationModel)


Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-23
Min 68 x[1,1,1] + 48 x[2,1,1] + 2 x[3,1,1] + 88 x[4,1,1] + 10 x[1,2,1] + 17 x[2,2,1] + 83 x[3,2,1] + 51 x[4,2,1] + 86 x[1,3,1] + 39 x[2,3,1] + 80 x[3,3,1] + 19 x[4,3,1] + 75 x[1,4,1] + 51 x[2,4,1] + 86 x[3,4,1] + 19 x[4,4,1] + 68 x[1,1,2] + 48 x[2,1,2] + 2 x[3,1,2] + 88 x[4,1,2] + 10 x[1,2,2] + 17 x[2,2,2] + 83 x[3,2,2] + 51 x[4,2,2] + 86 x[1,3,2] + 39 x[2,3,2] + 80 x[3,3,2] + 19 x[4,3,2] + 75 x[1,4,2] + 51 x[2,4,2] + 86 x[3,4,2] + 19 x[4,4,2] + 68 x[1,1,3] + 48 x[2,1,3] + 2 x[3,1,3] + 88 x[4,1,3] + 10 x[1,2,3] + 17 x[2,2,3] + 83 x[3,2,3] + 51 x[4,2,3] + 86 x[1,3,3] + 39 x[2,3,3] + 80 x[3,3,3] + 19 x[4,3,3] + 75 x[1,4,3] + 51 x[2,4,3] + 86 x[3,4,3] + 19 x[4,4,3] + 68 x[1,1,4] + 48 x[2,1,4] + 2 x[3,1,4] + 88 x[4,1,4] + 10 x[1,2,4] + 17 x[2,2,4] + 83 x[3,2,4] + 51 x[4,2,4] + 86 x[1,3,4] + 39 x[2,3,4] + 80 x[3,3,4] + 19 x[4,3,4] + 75 x[1,4,4] + 51 x[2,4,4] + 86 x[3,4,4] + 19 x[4,4,4]
Subject to


 p[1] : x[1,1,1] + x[1,2,1] + x[1,3,1] + x[1,4,1] + x[1,1,2] + x[1,2,2] + x[1,3,2] + x[1,4,2] + x[1,1,3] + x[1,2,3] + x[1,3,3] + x[1,4,3] + x[1,1,4] + x[1,2,4] + x[1,3,4] + x[1,4,4] == 1.0
 p[2] : x[2,1,1] + x[2,2,1] + x[2,3,1] + x[2,4,1] + x[2,1,2] + x[2,2,2] + x[2,3,2] + x[2,4,2] + x[2,1,3] + x[2,2,3] + x[2,3,3] + x[2,4,3] + x[2,1,4] + x[2,2,4] + x[2,3,4] + x[2,4,4] == 1.0
 p[3] : x[3,1,1] + x[3,2,1] + x[3,3,1] + x[3,4,1] + x[3,1,2] + x[3,2,2] + x[3,3,2] + x[3,4,2] + x[3,1,3] + x[3,2,3] + x[3,3,3] + x[3,4,3] + x[3,1,4] + x[3,2,4] + x[3,3,4] + x[3,4,4] == 1.0
 p[4] : x[4,1,1] + x[4,2,1] + x[4,3,1] + x[4,4,1] + x[4,1,2] + x[4,2,2] + x[4,3,2] + x[4,4,2] + x[4,1,3] + x[4,2,3] + x[4,3,3] + x[4,4,3] + x[4,1,4] + x[4,2,4] + x[4,3,4] + x[4,4,4] == 1.0
 c[1] : x[1,1,1] + x[2,1,1] + x[3,1,1] + x[4,1,1] + x[1,1,2] + x[2,1,2] + x[3,1,2] + x[4,1,2] + x[1,1,3] + x[2,1,3] + x[3,1,3] + x[4,1,3] + x[1,1,4] + x[2,1,4] + x[3,1,4] + x[4,1,4] == 1.0
 c[2] : x[1,2,1] + x[2,2,1] + x[3,2,1] + x[4,2,1] + x[1

Este bloque de código realiza la optimización del modelo que hemos definido previamente, es decir, encuentra la asignación óptima de profesores a cursos y aulas que minimiza el costo total.

optimize!(model): Esta función resuelve el modelo, ajustando las variables para minimizar el costo total de acuerdo con las restricciones.

@show termination_status(model): Muestra el estado de la optimización, indicando si el proceso fue exitoso, si hubo problemas, etc.

println("Total: ", objective_value(model)): Imprime el valor mínimo del costo total que se ha alcanzado con la asignación óptima.

El ciclo for: Itera a través de todas las combinaciones posibles de profesores (i), cursos (j) y aulas (k) para determinar qué asignaciones se han hecho. Si el valor de la variable x[i, j, k] es mayor que 0.5, se considera que el profesor i ha sido asignado al curso j en el aula k, y esto se imprime en la consola.

Esta version del prblema se llama axial.

In [ ]:
# Resolver el modelo de optimización para asignar profesores, cursos y aulas
optimize!(model)

# Verificar el estado final de la optimización
@show termination_status(model)

# Imprimir el costo total mínimo que el optimizador encontró
println("Costo Total: ", objective_value(model))

# Iterar para determinar qué asignaciones específicas se han realizado
for profesor = 1:N, curso = 1:N, aula = 1:N
    # Si la asignación es efectiva, se imprime
    if value(x[profesor, curso, aula]) > 0.5
        println("Profesor $profesor asignado al curso $curso en el aula $aula")
    end
end


termination_status(model) = MathOptInterface.OPTIMAL
Total: 70.0
 professor 1 assigned to course 2 at room 2
 professor 2 assigned to course 3 at room 1


 professor 3 assigned to course 1 at room 3
 professor 4 assigned to course 4 at room 4


Este bloque de código se encarga de resolver el modelo de optimización para la asignación de profesores a cursos y aulas, de forma que minimice el costo total. A continuación, detallo cada componente:

optimize!(model): Esta función se encarga de resolver el modelo de optimización. Esto significa que busca la combinación de asignaciones que minimice el costo total, respetando las restricciones impuestas.

@show termination_status(model): Muestra el estado final del proceso de optimización, lo cual es útil para entender si el modelo encontró una solución óptima o si hubo algún problema en el proceso.

println("Costo Total: ", objective_value(model)): Imprime el valor de la función objetivo, que en este contexto representa el costo total mínimo alcanzado con las asignaciones óptimas.

El bucle for: Este ciclo iterativo recorre todas las combinaciones posibles de profesor, curso y aula para identificar qué asignaciones se han hecho. Si el valor de la variable x[profesor, curso, aula] es mayor que 0.5, se considera que la asignación es efectiva y se imprime en la consola.

In [ ]:
using Gurobi  # Importar el módulo Gurobi para optimización
using JuMP  # Importar el módulo JuMP para modelado matemático

NumElementos = 7  # Número de elementos en cada dimensión (podría representar profesores, cursos y aulas)

# Crear un nuevo modelo de optimización usando Gurobi como optimizador
model = Model(Gurobi.Optimizer)

# Definir la variable de decisión 'asignacion'. Es una variable binaria tridimensional de tamaño NumElementos x NumElementos x NumElementos
@variable(model, asignacion[1:NumElementos, 1:NumElementos, 1:NumElementos], Bin)

# Definir la función objetivo para minimizar el costo total
@objective(model, Min, sum(cost .* asignacion))

# Restricciones para asegurar que cada combinación de profesor y curso se asigne a una sola aula
@constraint(model, ProfesorCurso[prof=1:NumElementos, curso=1:NumElementos], sum(asignacion[prof, curso, :]) == 1)

# Restricciones para asegurar que cada combinación de profesor y aula tenga un solo curso
@constraint(model, ProfesorAula[prof=1:NumElementos, aula=1:NumElementos], sum(asignacion[prof, :, aula]) == 1)

# Restricciones para asegurar que cada combinación de curso y aula tenga un solo profesor
@constraint(model, CursoAula[curso=1:NumElementos, aula=1:NumElementos], sum(asignacion[:, curso, aula]) == 1)

# Imprimir el modelo para verificar que se ha configurado correctamente
println(model)


Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-23


4×4 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 r[1,1] : x[1,1,1] + x[2,1,1] + x[3,1,1] + x[4,1,1] == 1.0  …  r[1,4] : x[1,1,4] + x[2,1,4] + x[3,1,4] + x[4,1,4] == 1.0
 r[2,1] : x[1,2,1] + x[2,2,1] + x[3,2,1] + x[4,2,1] == 1.0     r[2,4] : x[1,2,4] + x[2,2,4] + x[3,2,4] + x[4,2,4] == 1.0
 r[3,1] : x[1,3,1] + x[2,3,1] + x[3,3,1] + x[4,3,1] == 1.0     r[3,4] : x[1,3,4] + x[2,3,4] + x[3,3,4] + x[4,3,4] == 1.0
 r[4,1] : x[1,4,1] + x[2,4,1] + x[3,4,1] + x[4,4,1] == 1.0     r[4,4] : x[1,4,4] + x[2,4,4] + x[3,4,4] + x[4,4,4] == 1.0

Este es un modelo de optimización para un problema de asignación tridimensional. Supongamos que tenemos un número NumElementos de profesores, cursos, y aulas. Queremos asignar cada profesor a un curso y una aula de tal manera que minimicemos un "costo" total, que podría estar relacionado con la distancia entre las aulas, la afinidad del profesor con el curso, etc.

La variable asignacion[prof, curso, aula] es una variable binaria que será igual a 1 si el "profesor prof" está asignado al "curso curso" en la "aula aula", y 0 en caso contrario.

La matriz cost contiene los "costos" asociados con asignar el "profesor prof" al "curso curso" en la "aula aula".

Las restricciones garantizan que cada profesor se asigne a exactamente un curso y una aula, que cada curso tenga exactamente un profesor y se realice en un aula, y que en cada aula se realice exactamente un curso con un solo profesor.

La función objetivo busca minimizar el costo total de esta asignación.


In [ ]:
# Resolver el modelo de optimización. Esto ajusta las variables 'asignacion' para minimizar la función objetivo
# sujeto a las restricciones dadas.
optimize!(model)

# Mostrar el estado de terminación del modelo para saber si la optimización fue exitosa o no
@show termination_status(model)

# Imprimir el valor de la función objetivo, es decir, el costo total mínimo alcanzado
println("Costo Total Minimizado: ", objective_value(model))

# Imprimir los valores de las variables de decisión
println("Variables de Decisión: ", value.(asignacion))

# Iterar a través de todas las combinaciones de profesores (prof), cursos (curso), y aulas (aula)
# para encontrar qué asignaciones han sido realizadas por el optimizador.
for prof = 1:NumElementos, curso = 1:NumElementos, aula = 1:NumElementos
    # Si la variable 'asignacion[prof, curso, aula]' tiene un valor mayor que 0.5,
    # entonces la asignación es efectiva y se imprime.
    if value(asignacion[prof, curso, aula]) > 0.5
        println("Profesor $prof asignado al curso $curso en la aula $aula")
    end
end


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 48 rows, 64 columns and 192 nonzeros
Model fingerprint: 0x7c52da2d
Variable types: 0 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 936.0000000
Presolve time: 0.00s
Presolved: 48 rows, 64 columns, 192 nonzeros
Variable types: 0 continuous, 64 integer (64 binary)

Root relaxation: objective 6.240000e+02, 39 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     624.0000000  624.00000  0.00%     -    0s

Explored 1 nodes (39 simplex iterations) in 0.01 seconds (0.00 work units)
Thread co

 -0.0 1.0 0.0 -0.0; 1.0 -0.0 0.0 -0.0; -0.0 -0.0 1.0 0.0; 0.0 -0.0 -0.0 1.0;;; 1.0 -0.0 -0.0 -0.0; -0.0 1.0 0.0 -0.0; -0.0 0.0 -0.0 1.0; 0.0 0.0 1.0 -0.0;;; -0.0 0.0 -0.0 1.0; 0.0 -0.0 1.0 0.0; 0.0 1.0 -0.0 0.0; 1.0 -0.0 0.0 0.0]


  (1,1,3)  (1,2,2)  (1,3,1)  (1,4,4)  (2,1,2)  (2,2,3)  (2,3,4)  (2,4,1)  (3,1,1)  (3,2,4)  (3,3,2)  (3,4,3)  (4,1,4)  (4,2,1)  (4,3,3)  (4,4,2)

Este segmento de código resuelve el modelo de optimización que hemos definido previamente. Utiliza la función optimize!() para encontrar la combinación de asignaciones que minimiza el "costo" total, sujeto a las restricciones impuestas.

La línea @show termination_status(model) imprime el estado del modelo después de la optimización para verificar si la solución se ha encontrado exitosamente.

Posteriormente, el programa imprime el "Costo Total Minimizado", que es el valor de la función objetivo una vez que el modelo ha sido optimizado. Este valor representa el costo total mínimo que se puede alcanzar con las asignaciones óptimas.

Finalmente, el bucle for al final del código itera a través de todas las combinaciones posibles de "profesores", "cursos", y "aulas" para determinar qué asignaciones se han hecho. Las asignaciones efectivas (donde la variable asignacion[prof, curso, aula] es mayor a 0.5) se imprimen en la consola.

Una vez visto los modelos 3-dimensional, vamos a pasar a desarrollar los moedelos:

## Modelo de forma axial

#### Representamos el modelo de forma axial:


\begin{align*}
\min \sum_{i \in I} \sum_{j \in J} \sum_{k \in K} c_{ij}  x_{ij} & \text{Minimizar la función objetivo (Costo total)} \\
\text{s.t.:} & \text{Sujeto a las siguientes restricciones:} \\
& \sum_{i \in I} \sum_{j \in J} x_{ijk} = 1   &  \forall k \in K  \quad \text{Comentario 1: Cada elemento de K se asigna exactamente una vez} \\
& \sum_{j \in J} \sum_{k \in K} x_{ijk} = 1   &  \forall i \in I  \quad \text{Comentario 2: Cada elemento de I se asigna exactamente una vez} \\
& \sum_{i \in I} \sum_{k \in K} x_{ijk} = 1   &  \forall j \in J  \quad \text{Comentario 3: Cada elemento de J se asigna exactamente una vez} \\
& x_{ijk} \in \{ 0,1 \}                       & \forall i \in I,  j \in J,  k \in K  \quad \text{Comentario 4: Variables binarias (Asignación)} \\
\end{align*}


## Código en Julia para el modelo

In [6]:
using JuMP
using GLPK

# Conjuntos de índices
I = 1:3
J = 1:3
K = 1:3

# Costos de asignación
c = [5 2 7
     3 1 4
     6 8 9]

# Crear el modelo de optimización
model = Model(optimizer_with_attributes(GLPK.Optimizer, "msg_lev" => GLPK.GLP_MSG_OFF))

# Variables binarias que indican las asignaciones
@variable(model, x[i in I, j in J, k in K], Bin)

# Función objetivo: Minimizar el costo total
@objective(model, Min, sum(c[i, j] * x[i, j, k] for i in I, j in J, k in K))

# Restricciones: Cada elemento de K se asigna exactamente una vez
@constraint(model, [k in K], sum(x[i, j, k] for i in I, j in J) == 1)

# Restricciones: Cada elemento de I se asigna exactamente una vez
@constraint(model, [i in I], sum(x[i, j, k] for j in J, k in K) == 1)

# Restricciones: Cada elemento de J se asigna exactamente una vez
@constraint(model, [j in J], sum(x[i, j, k] for i in I, k in K) == 1)

# Resolver el modelo
optimize!(model)

# Verificar el estado de la solución
if termination_status(model) == MOI.OPTIMAL
    println("Solución óptima encontrada:")
    for k in K
        println("Asignaciones para k =", k)
        for i in I
            for j in J
                if value(x[i, j, k]) > 0.5
                    println("x[$i, $j, $k] = ", value(x[i, j, k]))
                end
            end
        end
    end
else
    println("No se encontró una solución óptima.")
end


Solución óptima encontrada:
Asignaciones para k =1
x[2, 3, 1] = 1.0
Asignaciones para k =2
x[3, 1, 2] = 1.0
Asignaciones para k =3
x[1, 2, 3] = 1.0


## Modelo de forma planar

#### Representamos el modelo de forma planar:


\begin{align*}
\min \sum_{i \in I} \sum_{j \in J} \sum_{k \in K} c_{ij}  x_{ij} & \text{Minimizar la función objetivo (Costo total)} \\
\text{s.t.:} & \text{Sujeto a las siguientes restricciones:} \\
& \sum_{k \in K} x_{ijk} = 1   &  \forall i \in I, \forall j \in J  \quad \text{Comentario 1: Cada (i, j) se asigna exactamente a un k} \\
& \sum_{i \in I} x_{ijk} = 1   &  \forall j \in J, \forall k \in K  \quad \text{Comentario 2: Cada (j, k) se asigna exactamente a un i} \\
& \sum_{j \in J} x_{ijk} = 1   &  \forall i \in I, \forall k \in K  \quad \text{Comentario 3: Cada (i, k) se asigna exactamente a un j} \\
& x_{ijk} \in \{ 0,1 \}                       & \forall i \in I,  j \in J,  k \in K  \quad \text{Comentario 4: Variables binarias (Asignación)} \\
\end{align*}


## Código en Julia

In [7]:
using JuMP
using GLPK

# Conjuntos de índices
I = 1:3
J = 1:3
K = 1:3

# Costos de asignación
c = [5 2 7
     3 1 4
     6 8 9]

# Crear el modelo de optimización
model = Model(optimizer_with_attributes(GLPK.Optimizer, "msg_lev" => GLPK.GLP_MSG_OFF))

# Variables binarias que indican las asignaciones
@variable(model, x[i in I, j in J, k in K], Bin)

# Función objetivo: Minimizar el costo total
@objective(model, Min, sum(c[i, j] * x[i, j, k] for i in I, j in J, k in K))

# Restricciones: Cada elemento de I se asigna exactamente una vez
@constraint(model, [i in I, j in J], sum(x[i, j, k] for k in K) == 1)

# Restricciones: Cada elemento de J se asigna exactamente una vez
@constraint(model, [j in J, k in K], sum(x[i, j, k] for i in I) == 1)

# Restricciones: Cada elemento de K se asigna exactamente una vez
@constraint(model, [i in I, k in K], sum(x[i, j, k] for j in J) == 1)

# Resolver el modelo
optimize!(model)

# Verificar el estado de la solución
if termination_status(model) == MOI.OPTIMAL
    println("Solución óptima encontrada:")
    for k in K
        println("Asignaciones para k =", k)
        for i in I
            for j in J
                if value(x[i, j, k]) > 0.5
                    println("x[$i, $j, $k] = ", value(x[i, j, k]))
                end
            end
        end
    end
else
    println("No se encontró una solución óptima.")
end


Solución óptima encontrada:
Asignaciones para k =1
x[1, 1, 1] = 1.0
x[2, 3, 1] = 1.0
x[3, 2, 1] = 1.0
Asignaciones para k =2
x[1, 2, 2] = 1.0
x[2, 1, 2] = 1.0
x[3, 3, 2] = 1.0
Asignaciones para k =3
x[1, 3, 3] = 1.0
x[2, 2, 3] = 1.0
x[3, 1, 3] = 1.0



# Conclusión y Aplicaciones Prácticas

## Resumen de la Implementación
A lo largo de este trabajo, hemos implementado un modelo de optimización tridimensional usando programación lineal entera. El modelo utiliza una matriz tridimensional de variables binarias para representar asignaciones a lo largo de tres ejes: "plano", "axial", y otra dimensión que podríamos llamar "vertical". Estos ejes permiten considerar múltiples aspectos en la toma de decisiones, como asignar profesores a cursos y aulas en nuestro caso particular.

#Aplicaciones en Diferentes Ámbitos
El potencial del modelo de optimización tridimensional va más allá de la educación y se extiende a diversos campos. A continuación, se presentan algunas áreas en las que este modelo podría tener un impacto significativo:

##Logística
- Distribución de Mercancías: Podría usarse para asignar camiones a diferentes rutas y horarios, teniendo en cuenta variables como el consumo de combustible, distancia y tiempos de entrega.

- Gestanterías podría minimizar el tiempo de recuperación y maximizar el espacio disponible.

- Planificación de Flotas: Asignar vehículos a distintas bases o puertos de origen, considerando factores como mantenimiento, disponibilidad y costos operativos.

##Salud
- Gestión Hospitalaria: Podría ayudar a asignar personal médico, camas y equipos a diferentes departamentos o aulas de un hospital, teniendo en cuenta horarios, especialidades y urgencias.

- Optimización de Horarios de Cirugía: Podría utilizarse para asignar quirófanos, equipos médicos y horarios a distintas cirugías, buscando minimizar los tiempos de espera y optimizar el uso de los recursos.

- Telemedicina: En una red de atención médica en línea, el modelo podría asignar pacientes a médicos en función de la especialización, disponibilidad y ubicación geográfica.

##Industria
- Manufactura: Asignar trabajadores a líneas de ensamblaje y máquinas, teniendo en cuenta la experiencia del trabajador, la dificultad de la tarea y las limitaciones de tiempo.

- Gestión de la Cadena de Suministro: Asignar proveedores a plantas de fabricación y centros de distribución, considerando factores como costos, calidad y tiempos de entrega.

- Optimización Energética: En una planta industrial, asignar diferentes fuentes de energía a maquinaria y procesos, teniendo en cuenta la eficiencia energética y los costos operativos.

Cada uno de estos ejemplos muestra cómo un modelo tridimensional de programación lineal entera puede manejar múltiples variables y criterios para ofrecer soluciones más completas y eficientes en una amplia gama de industrias.

# Ventajas de un Modelo Tridimensional
- Múltiples Criterios: Permite considerar simultáneamente varios aspectos, mejorando la calidad de la solución.

- Flexibilidad: Puede adaptarse fácilmente a cambios en las dimensiones o en el número de categorías.

- Eficiencia Computacional: Los optimizadores modernos pueden resolver este tipo de modelos en tiempos razonables incluso para grandes valores de N.



# Limitaciones y Futuras Líneas de Trabajo
##Limitaciones
-Calidad de Datos: Uno de los mayores desafíos es la precisión y la calidad de los datos de entrada. Un modelo es tan bueno como los datos en los que se basa. Datos erróneos o incompletos pueden llevar a soluciones subóptimas o incluso inviables.

- Escalabilidad: A medida que aumenta el tamaño del problema (por ejemplo, más asignaciones o categorías), la complejidad computacional podría convertirse en un obstáculo.

- Adaptabilidad: Cambios en las restricciones o en los objetivos del modelo pueden requerir reajustes significativos en el modelo.

- Recursos Computacionales: Algoritmos de optimización más avanzados pueden ser computacionalmente costosos, lo que requiere hardware especializado.

##Futuras Líneas de Trabajo
Algoritmos Heurísticos: Explorar métodos aproximados que puedan ofrecer soluciones "suficientemente buenas" en un tiempo más corto.

- Machine Learning: Integrar técnicas de aprendizaje automático para predecir y ajustar dinámicamente los costos y restricciones.

-Interfaz de Usuario: Desarrollar una interfaz amigable que permita a los usuarios no técnicos configurar y resolver problemas de asignación de manera más fácil.

-Optimización Multiobjetivo: Adaptar el modelo para manejar múltiples objetivos, lo que permitiría una toma de decisiones más completa y equilibrada.

##Conclusión Extendida
La programación lineal entera tridimensional ofrece un marco robusto y flexible para abordar problemas complejos de asignación en una variedad de campos. Desde la logística y la salud hasta la industria manufacturera, este modelo tiene el potencial de revolucionar la forma en que tomamos decisiones estratégicas y operativas.

Sin embargo, es vital tener en cuenta sus limitaciones, especialmente en términos de calidad de datos y escalabilidad. Las futuras líneas de trabajo pueden abordar estas cuestiones a través de la incorporación de algoritmos heurísticos, técnicas de aprendizaje automático y optimización multiobjetivo.

Este modelo es una herramienta poderosa, pero su éxito final dependerá de cómo se adapte y se aplique a problemas específicos y de cuán precisos y completos sean los datos en los que se basa. Estamos solo al comienzo de explorar todo su potencial y las oportunidades para mejoras y aplicaciones futuras son abundantes.





